# Conditions

* signal a condtion
* handle it
* restart

In [1]:
(proclaim '(optimize (debug 3)
                   (safety 3)
                   (speed 0)))

In [ ]:
; (error "SOMETHING WRONG HAPPENED.")

SIMPLE-ERROR: SOMETHING WRONG HAPPENED.



SIMPLE-ERROR: SOMETHING WRONG HAPPENED.

# Examples in PCL

In [ ]:
;;; log parse facilities
(defclass log-entry ()
    ((text
      :initarg :text)))

; a flag for mocking malformed log entry
(defparameter *text-form-flag* nil)

(defun well-formed-log-entry-p (text)
  (declare (ignore text))
  *text-form-flag*)

#<STANDARD-CLASS COMMON-LISP-USER::LOG-ENTRY>

*TEXT-FORM-FLAG*

WELL-FORMED-LOG-ENTRY-P

## Conditions

* define-condition
* make-instance

In [ ]:
(define-condition malformed-log-entry-error (error)
    ((text :initarg :text :reader text))
  (:report (lambda (condition stream) (format stream "~A" (text condition)))))

(defun parse-log-entry (text)
  (if (well-formed-log-entry-p text)
      (make-instance 'log-entry :text text)
      ;; signal an error
      (error 'malformed-log-entry-error :text text)))


MALFORMED-LOG-ENTRY-ERROR

PARSE-LOG-ENTRY

SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::TEXT


## Condition Handlers

* handler-case

In [4]:
;;; without condition handlers
(defun parse-log-file (file)
  (with-open-file (in file :direction :input)
    (loop for text = (read-line in nil nil) while text
          for entry = (parse-log-entry text)
            when entry collect it)))

PARSE-LOG-FILE

In [5]:
; (parse-log-file "a.log")

; MALFORMED-LOG-ENTRY-ERROR: A quick brown fox jumps over the lazy dog.

In [6]:
;;; with condition handlers
(defun parse-log-file (file)
  (with-open-file (in file :direction :input)
    (loop for text = (read-line in nil nil) while text
          for entry = (handler-case (parse-log-entry text) ;; handler
                        (malformed-log-entry-error () 'malformed-log-entry-value))
            when entry collect it)))

PARSE-LOG-FILE

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::PARSE-LOG-FILE in DEFUN


In [7]:
(trace parse-log-entry)
(parse-log-file "a.log")
(untrace parse-log-entry)

(PARSE-LOG-ENTRY)

  0: (PARSE-LOG-ENTRY "A quick brown fox jumps over the lazy dog.")
  0: PARSE-LOG-ENTRY exited non-locally
  0: (PARSE-LOG-ENTRY "A quick brown fox jumps over the lazy dog.")
  0: PARSE-LOG-ENTRY exited non-locally


(MALFORMED-LOG-ENTRY-VALUE MALFORMED-LOG-ENTRY-VALUE)

T

## Restarts

* restart-case
* handler-bind

In [ ]:
(defun skip-log-entry (c) ;; the condition handler function
  (declare (ignore c))
  (let ((restart (find-restart 'skip-log-entry)))
    ;; invoke restart
    (when restart (invoke-restart restart))))

(defun analyze-entry (entry)
  (format t "Analyze entry: ~A(~A)~%" (slot-value entry 'text) (type-of entry)))

; redefine function
(defun parse-log-file (file)
  (with-open-file (in file :direction :input)
    (loop for text = (read-line in nil nil) while text
          for entry = (restart-case (parse-log-entry text)
                        ;; restart
                        (skip-log-entry () (format t "parse-log-entry FAILED.")))
            when entry collect it)))

(defun analyze-log (log)
  (dolist (entry (parse-log-file log))
    (analyze-entry entry)))

(defun find-all-logs ()
  (list "a.log"))

(defun log-analyzer ()
  ;; bind condition handler
  (handler-bind ((malformed-log-entry-error #'skip-log-entry))
    (dolist (log (find-all-logs))
      (analyze-log log))))

SKIP-LOG-ENTRY

ANALYZE-ENTRY

PARSE-LOG-FILE

ANALYZE-LOG

FIND-ALL-LOGS

LOG-ANALYZER

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::PARSE-LOG-FILE in DEFUN


In [9]:
(trace find-all-logs analyze-log parse-log-file parse-log-entry analyze-entry)
(log-analyzer)
(untrace find-all-logs analyze-log parse-log-file parse-log-entry analyze-entry)


(FIND-ALL-LOGS ANALYZE-LOG PARSE-LOG-FILE PARSE-LOG-ENTRY ANALYZE-ENTRY)

  0: (FIND-ALL-LOGS)
  0: FIND-ALL-LOGS returned ("a.log")
  0: (ANALYZE-LOG "a.log")
    1: (PARSE-LOG-FILE "a.log")
      2: (PARSE-LOG-ENTRY "A quick brown fox jumps over the lazy dog.")
      2: PARSE-LOG-ENTRY exited non-locally
parse-log-entry FAILED.      2: (PARSE-LOG-ENTRY "A quick brown fox jumps over the lazy dog.")
      2: PARSE-LOG-ENTRY exited non-locally
parse-log-entry FAILED.    1: PARSE-LOG-FILE returned NIL
  0: ANALYZE-LOG returned NIL


NIL

T

In [10]:
(trace find-all-logs analyze-log parse-log-file parse-log-entry analyze-entry)
(let ((*text-form-flag* t))
  (log-analyzer))
(untrace find-all-logs analyze-log parse-log-file parse-log-entry analyze-entry)

(FIND-ALL-LOGS ANALYZE-LOG PARSE-LOG-FILE PARSE-LOG-ENTRY ANALYZE-ENTRY)

  0: (FIND-ALL-LOGS)
  0: FIND-ALL-LOGS returned ("a.log")
  0: (ANALYZE-LOG "a.log")
    1: (PARSE-LOG-FILE "a.log")
      2: (PARSE-LOG-ENTRY "A quick brown fox jumps over the lazy dog.")
      2: PARSE-LOG-ENTRY returned #<LOG-ENTRY {1102537133}>
      2: (PARSE-LOG-ENTRY "A quick brown fox jumps over the lazy dog.")
      2: PARSE-LOG-ENTRY returned #<LOG-ENTRY {11025376D3}>
    1: PARSE-LOG-FILE returned
         (#<LOG-ENTRY {1102537133}> #<LOG-ENTRY {11025376D3}>)
    1: (ANALYZE-ENTRY #<LOG-ENTRY {1102537133}>)
Analyze entry: A quick brown fox jumps over the lazy dog.(LOG-ENTRY)
    1: ANALYZE-ENTRY returned NIL
    1: (ANALYZE-ENTRY #<LOG-ENTRY {11025376D3}>)
Analyze entry: A quick brown fox jumps over the lazy dog.(LOG-ENTRY)
    1: ANALYZE-ENTRY returned NIL
  0: ANALYZE-LOG returned NIL


NIL

T

## Multiple Restarts

In [ ]:
; redefine function
(defun parse-log-entry (text)
  (if (well-formed-log-entry-p text)
      (make-instance 'log-entry :text text)
      (restart-case (error 'malformed-log-entry-error :text text)
        ;; standard restart
        (use-value (value) value)
        ;; customed restart
        (reparse-entry (fixed-text)
                       (parse-log-entry fixed-text)
                       ; mock need fixing
                       (setf *text-form-flag* nil)))))


(defclass malformed-log-entry (log-entry)
    ())

(defun log-analyzer ()
  ;; bind condition handler
  (handler-bind ((malformed-log-entry-error
                  #'(lambda (c)
                      (if (find-restart 'reparse-entry)
                          (progn
                           (setf *text-form-flag* t) ; mock fixed
                           (invoke-restart 'reparse-entry (read-line)))
                          (use-value (make-instance 'malformed-log-entry :text (text c)))))))
    (dolist (log (find-all-logs))
      (analyze-log log))))

PARSE-LOG-ENTRY

#<STANDARD-CLASS COMMON-LISP-USER::MALFORMED-LOG-ENTRY>

LOG-ANALYZER

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::PARSE-LOG-ENTRY in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::LOG-ANALYZER in DEFUN


In [12]:
(trace find-all-logs analyze-log parse-log-file parse-log-entry analyze-entry)
(log-analyzer)
(untrace find-all-logs analyze-log parse-log-file parse-log-entry analyze-entry)

(FIND-ALL-LOGS ANALYZE-LOG PARSE-LOG-FILE PARSE-LOG-ENTRY ANALYZE-ENTRY)

  0: (FIND-ALL-LOGS)
  0: FIND-ALL-LOGS returned ("a.log")
  0: (ANALYZE-LOG "a.log")
    1: (PARSE-LOG-FILE "a.log")
      2: (PARSE-LOG-ENTRY "A quick brown fox jumps over the lazy dog.")
        3: (PARSE-LOG-ENTRY "fixed text")
        3: PARSE-LOG-ENTRY returned #<LOG-ENTRY {110288A243}>
      2: PARSE-LOG-ENTRY returned NIL
      2: (PARSE-LOG-ENTRY "A quick brown fox jumps over the lazy dog.")
        3: (PARSE-LOG-ENTRY "fixed text2")
        3: PARSE-LOG-ENTRY returned #<LOG-ENTRY {110288B3D3}>
      2: PARSE-LOG-ENTRY returned NIL
    1: PARSE-LOG-FILE returned NIL
  0: ANALYZE-LOG returned NIL


NIL

T